<a id="bql-top"></a>
<div style="width: 50px; height: 50px; border-radius: 50%; background: #27AE60; margin-bottom: 0px;">
    <img src="../img/Welcome Page/boot_camp.png">
</div>
<h1 style="margin-top: 20px; margin-bottom: 5px;">The Bloomberg Query Language (BQL)<br><span style="color: orange; margin-bottom: 0px;">Use Python to Access Bloomberg Data</span></h1>
<h4 style="color: orange; margin-top: 0px;">(25-min Read)</h4>

By the end of this document, you will be able to:
1. Explain the basics of how the `bql` library and `bql.Service()` object work
1. Build and execute a BQL request to get Bloomberg data into your project
1. Understand the basics of a `pandas` DataFrame - a fundamental structure for data analysis
1. Use the BQL Editor to prepare BQL requests

### Contents
<ul style="margin-bottom: 0px;">
    <li><a href="#project-flow-bql">The BQuant Project Flow</a></li>
    <li><a href="#bql-lib">The <code>bql</code> Python library</a></li>
        <ul>
            <li><a href="#bql-service">The <code>bql.Service()</code> Object</a></li>
            <li><a href="#bql-univ">BQL Universes, Data, and Functions</a></li>
            <li><a href="#bql-req">BQL Requests and Responses</a></li>
            <li><a href="#bql-res">Parsing a BQL Response</a></li>
        </ul>
    <li><a href="#bql-pandas">Basics of the <code>pandas</code> DataFrame</a></li>
    <ul>
        <li><a href="#df-index">DataFrame Index Column</a></li>
        <li><a href="#df-ops">Manipulate the DataFrame</a></li>
        <li><a href="#df-concat">Concatenate, Merge, and Pivot</a></li>
    </ul>
    <li><a href="#bql-editor">Using the BQL Editor Tool</a></li>
</ul>
<br>
<h5 style="margin-top: 0px;">&emsp;Examples</h5>
<ul>
    <li><a href="#bql-example">Example: One Universe, One Data Item</a></li>
    <li><a href="#bql-example2">Example: One Universe, Multiple Data Items</a></li>
    <li><a href="#bql-example3">Example: Multiple Universes, One Data Item</a></li>
    <li><a href="#bql-example4">Example: Multiple Universes, Multiple Data Items</a></li>
</ul>

<a href="2 Python Basics.ipynb">&larr; Back to Python Basics</a>&emsp; | &emsp;
<a href="4 Boot Camp Project.ipynb">Continue to Boot Camp: Final Project &rarr;</a>

---
<a id="project-flow-bql"></a>
<h1>BQuant Project Flow<br><span style="color: orange;">5-Step Process for Developing Your Projects</span></h1>

The 5-step project flow for BQuant development was mentioned in the Intro to BQuant section, but it bears repeating here. Every project you develop in BQuant will follow the same basic process below.

1. Setup Environment
1. Get Data from Bloomberg using BQL
1. Additional Data Munging (if necessary)
1. User Interface & Visualizations
1. Simplify, Document, & Publish

In this notebook, we cover the use of the Bloomberg Query Language (BQL) in Steps 1 and 2.

[&uarr; Return to Top](#bql-top)

---
<a id="bql-lib"></a>
<h1>The <code>bql</code> Python Library<br><span style="color: orange;">Introduction</span></h1>

BQL is the main tool for retrieving data in the BQuant environment. In order to use BQL, your project needs access to the objects and functions stored in the `bql` library. Recall from the Intro to BQuant section that <a href="1 Intro to BQuant.ipynb#intro-1-5">Python libraries</a> are pre-written code files stored separately from your project. Engineers at Bloomberg created the `bql` library specifically for BQuant, and you can access it with an `import bql` statement at the top of your project.

In [1]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
import bql

&#9888; A quick note on the `import` statement. It's considered best practice in any Python/Jupyter file to import all the necessary libraries for the entire project at the top. As this is a tutorial, we will show the `import bql` statement commented out in each code cell to display the entire block of code necessary to carry out a task. In your projects, however, you will only need to call `import bql` one time, in one cell at the top of the notebook.

[&uarr; Return to Top](#bql-top)

<a id="bql-service"></a>
<h1><span style="color: orange;">The <code style="background-color: transparent; color: orange;">bql.Service()</code> Object</span></h1>

Inside the `bql` library is an object called `Service`. You will use this service object to connect to Bloomberg's servers and send and receive requests for data. After you import the `bql` library, declare a variable to store the `bql.Service()` object. You can call this variable whatever you want, but the convention among most BQuant developers is to name it `bq`, so let's use that name.

In [2]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
bq = bql.Service()

&#9888; The same principle applies to the `bql.Service()` object: you only need to declare it once at the top of the project. We will comment out this code in the Python code cells that follow.

[&uarr; Return to Top](#bql-top)

<a id="bql-univ"></a>
<h1><span style="color: orange;">BQL Universes, Fields, and Functions</span></h1>

When making a BQL request for data, you will need to pass two arguments: the type of data, and the universe. You can think of BQL queries as having two statements: `GET` and `FOR`. If you need daily trading volume for Apple stock, you are asking BQL to "get" volume data "for" Apple. All BQL queries will follow this basic structure: `GET <data type> FOR <universe>`.

While the universe variable is fairly straightforward, the data type can get quite complex. This data type variable can be a combination of BQL "fields" and "functions" for the universe. For example, if you are only interested in the daily volume, you can simply ask BQL to retrieve one field (volume). But suppose you are interested in the average volume over a period of time. In this case, you would need to perform a function (average) on the field (volume). Oftentimes you will need a combination of fields and functions to get the right data.

The `bql.Service()` object, normally named `bq`, gives you access to these fields and functions with the syntax `bq.field` and `bq.func`, respectively. These fields and functions can be chained together to perform calculations. The example below prepares two data types for a BQL request: `vol` for the daily trading volume for Apple stock over the last 30 days, and `avg_vol` for the average of those numbers. Running the cell below will not produce any output; we'll discuss how to make a request and parse a response in the next sections.

In [3]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get data with BQL
universe = 'AAPL US Equity'                         # <-- Bloomberg ticker for the security
vol = bq.data.px_volume('-30D', '0D')               # <-- use bq.data.px_volume() to retrieve volume data
avg_vol = vol.avg()                                 # <-- use a BQL function (bq.func.avg) to retrieve the average of the vol time series

[&uarr; Return to Top](#bql-top)

<a id="bql-req"></a>
<h1><span style="color: orange;">BQL Requests and Responses</span></h1>

Once a universe and data type have been declared, it's time to write a BQL request, send it to Bloomberg servers, and receive a response. Use the `bql.Request` object to initiate the request, and the `bq.Service` object has an `execute` method that will send the request and receive a response.

In [4]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get data with BQL
universe = 'AAPL US Equity'            # <-- Bloomberg ticker for the security
vol = bq.data.px_volume('-30D', '0D')  # <-- use bq.data.px_volume() to retrieve volume data
req = bql.Request(universe, vol)       # <-- bql Request object, pass the universe first, then data
res = bq.execute(req)                  # <-- use the bql.Service() object to execute the request and get a response

# get the length of the response
len(res)

1

Notice that this response object `res` has a length of 1. This is because we requested only one dataset - `vol`. In the example below, we pass a list to the `bql.Request` object with two datasets for the same universe. Now the length of the response object is 2.

In [5]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get data with BQL
universe = 'AAPL US Equity'
vol = bq.data.px_volume('-30D', '0D')         # <-- last 30 days of volume
price = bq.data.px_last('-30D', '0D')         # <-- last 30 days of prices
req = bql.Request(universe, [price, vol])     # <-- pass a list as the data attribute
res = bq.execute(req)

# get the length of the response
len(res)

2

Each item in the response object has a `name` property. By default, the name of each response item is a string version of the query you wrote in Python. See the example below.

In [6]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get data with BQL
universe = 'AAPL US Equity'
vol = bq.data.px_volume('-30D', '0D')         # <-- last 30 days of volume
price = bq.data.px_last('-30D', '0D')         # <-- last 30 days of prices
req = bql.Request(universe, [price, vol])     # <-- pass a list as the data attribute
res = bq.execute(req)

# print the name of each response item
for item in res:
    print(item.name)

PX_LAST(-30D,0D)
PX_VOLUME(-30D,0D)


For complex queries, these names could stretch to several hundred characters. We can control the names of these items by passing a dictionary to the `bql.Request` object instead of a list. These names will be important when we extract the data in the next section.

In [7]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get data with BQL
universe = 'AAPL US Equity'
vol = bq.data.px_volume('-30D', '0D')         # <-- last 30 days of volume
price = bq.data.px_last('-30D', '0D')         # <-- last 30 days of prices
req = bql.Request(universe, {'Price': price,  # <-- pass a dictionary as the data attribute
                             'Volume': vol})     
res = bq.execute(req)

# print the name of each response item
for item in res:
    print(item.name)

Price
Volume


[&uarr; Return to Top](#bql-top)

<a id="bql-res"></a>
<h1><span style="color: orange;">Parsing a BQL Response</span></h1>

So far we've explored some of the properties of the `bql.Response` object, but we haven't seen the actual data we requested. In order to use the data in the response object, we need to call the `df()` function on the response items. This function will extract the data into a DataFrame: a spreadsheet-like object that we'll discuss later in this notebook.

The example below will extract the data into a DataFrame, and print the first 5 rows to the screen. Notice that the column header is the same as the key in our dictionary ("Volume").

In [8]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Get data with BQL
universe = 'AAPL US Equity'                        # <-- Bloomberg ticker for the security
avg_vol = bq.data.px_volume('-30D', '0D')          # <-- chain together 'bq.data.px_volume' and 'bq.func.avg'
req = bql.Request(universe, {'Volume': avg_vol})   # <-- use a dictionary to change the column heading to 'Average 30D Volume'
res = bq.execute(req)                              # <-- use the bql.Service() object to execute the request and get a response
df = res[0].df()                                   # <-- parse the first item in the response as a DataFrame using the df() method
df.head()                                          # <-- display the first 5 rows of the DataFrame to the screen



,DATE,Volume
ID,,
AAPL US Equity,2021-03-09,129525780.0
AAPL US Equity,2021-03-10,111943326.0
AAPL US Equity,2021-03-11,103026514.0
AAPL US Equity,2021-03-12,88105050.0
AAPL US Equity,2021-03-13,NaN


<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice building, executing, and parsing a BQL Request
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Import the <code>bql</code> library and establish a connection with the <code>bql.Service()</code> object.</li>
            <li>Set a universe variable to the Bloomberg Americas-World Index <code>'BWORLDUS Index'</code></li>
            <li>Declare a variable named <code>price</code> and set it equal to <code>bq.data.px_last('-30D', '0D')</code> to retrieve prices for the last 30 days.</li>
            <li>Declare a variable named <code>req</code> and set it equal to a <code>bql.Request</code> object using the universe and price variables declared in Steps 3 and 4. Use a dictionary to control the name of the data item.</li>
            <li>Declare a variable named <code>res</code> and set it equal to the <code>bq.execute()</code> method to execute the BQL request and receive a response.</li>
            <li>Extract the data from <code>res</code> into a DataFrame using the <code>df()</code> method on the first item in the response.</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# Step 2
# import bql
# bq = bql.Service()

# Step 3
universe = 'BWORLDUS Index'

# Step 4
price = bq.data.px_last('-30D', '0D')

# Step 5
req = bql.Request(universe, {'Closing Price': price})

# Step 6
res = bq.execute(req)

# Step 7
df = res[0].df()

# show the first 5 rows
df.head()

[&uarr; Return to Top](#bql-top)

---
<a id="bql-pandas"></a>
<h1>Basics of the <code>pandas</code> DataFrame<span style="color: orange;"><br>Using DataFrames for Analysis</span></h1>

When we call the `df()` method on a BQL response object, the data inside is unpacked into a row/column format that resembles an Excel spreadsheet. In Python, this row/column object is called a "DataFrame." These DataFrames were invented by an engineer at AQR Capital Management and made public in 2009. Today any Python developer can access these DataFrame objects via an open-source library called `pandas`. The `pandas` library and its DataFrames are foundational to most data analysis projects in Python.

To fully understand the data we're working with in BQL, we'll need a basic understanding of how these DataFrames work. The following code cells will cover the basics of building and manipulating a DataFrame for use in BQL.

<br>
<h4 style="color: orange;">Build a Simple DataFrame</h4>
In the example below, we build a simple DataFrame and display it as a Jupyter cell output.

In [45]:
# import the pandas library
import pandas as pd

# build a 2-column DataFrame by using a dictionary
df = pd.DataFrame(data={'column 1': [1, 2, 3, 4],
                        'column 2': [100, 200, 300, 400]})

# Jupyter cell output
df

,column 1,column 2
0,1,100
1,2,200
2,3,300
3,4,400


[&uarr; Return to Top](#bql-top)

<a id="df-index"></a>
<h1><span style="color: orange;">DataFrame Index Column</span></h1>

Notice that the DataFrame automatically generated a column to the left that we didn't specify in our dictionary.
<br><br>
<img src="../img/Boot Camp/df_index.png">
<p>This column is the DataFrame's "index" which is responsible for uniquely identifying each row. A DataFrame index can be any list of values so long as the list is the same length as the DataFrame. We can also name the index similar to the way we named the columns. See the example below.</p>

In [46]:
# import the pandas library
# import pandas as pd

# build a 2-column DataFrame with a custom index
df = pd.DataFrame(data={'column 1': [1, 2, 3, 4],
                        'column 2': [100, 200, 300, 400]},
                  index=['ind1', 'ind2', 'ind3', 'ind4'])

# name the index
df.index.name = 'My Index'

# show the DataFrame
df

,column 1,column 2
My Index,,
ind1,1,100
ind2,2,200
ind3,3,300
ind4,4,400


<br>
<h4 style="color: orange;">Reset a DataFrame's Index</h4>
<p>BQL will automatically assign a list of ticker IDs to the DataFrame index. On occasion you may need to move this column from the index to use it like any other column in the DataFrame. DataFrames have a <code>reset_index</code> method that will move the current index over into a column and establish a new default index.</p>

In [47]:
# import the pandas library
# import pandas as pd

# build a 2-column DataFrame with a custom index
df = pd.DataFrame(data={'column 1': [1, 2, 3, 4],
                        'column 2': [100, 200, 300, 400]})
df.index.name = 'Original Index'

# reset the index
df = df.reset_index()

# show the DataFrame
df

,Original Index,column 1,column 2
0,0,1,100
1,1,2,200
2,2,3,300
3,3,4,400


[&uarr; Return to Top](#bql-top)

<a id="df-ops"></a>
<h1><span style="color: orange;">Manipulate the DataFrame</span></h1>
Run each of the code cells below to see some of the basic functions of a DataFrame.

In [68]:
# import the pandas library
# import pandas as pd

# build a 2-column DataFrame with a custom index
df = pd.DataFrame(data={'column 1': [1, 2, 3, 4],
                        'column 2': [100, 200, 300, 400]})



<h4 style="color: orange;">Get a List of Column Names</h4>

In [69]:
list(df.columns)

['column 1', 'column 2']

<h4 style="color: orange;">Select the Columns</h4>

In [70]:
# get one column
# use a list of column names to extract a smaller DataFrame
col_1 = df[['column 1']]

# show col_1
col_1

,column 1
0,1
1,2
2,3
3,4


In [71]:
# multiple columns
col1_and_col2 = df[['column 1', 'column 2']]  # <-- pass a list of 2 column names

# show col1_and_col2
col1_and_col2

,column 1,column 2
0,1,100
1,2,200
2,3,300
3,4,400


In [72]:
# get a single column as a list of values
col1_list = df['column 1'].tolist()

# show the list
col1_list

[1, 2, 3, 4]

<h4 style="color: orange;">Add New Columns</h4>

In [51]:
# add a new column by passing a list of data
# name the new column 'column 3'
df['column 3'] = [1000, 2000, 3000, 4000]

# show the dataframe
df

,column 1,column 2,column 3
0,1,100,1000
1,2,200,2000
2,3,300,3000
3,4,400,4000


<h4 style="color: orange;">Column Operations</h4>

In [52]:
# add a new column by adding two columns
df['col1 + col2'] = df['column 1'] + df['column 2']   # <-- use single brackets; df['column 1'] + df['column 2']

# add a new column by exponentiation
df['col2 ^ col1'] = df['column 2'] ** df['column 1']

# show df
df

,column 1,column 2,column 3,col1 + col2,col2 ^ col1
0,1,100,1000,101,100
1,2,200,2000,202,40000
2,3,300,3000,303,27000000
3,4,400,4000,404,25600000000


<h4 style="color: orange;">Select Individual Cells</h4>

In [53]:
# get the value in the first cell of column 2
df['column 2'][0]

100

<h4 style="color: orange;">Change Individual Cells</h4>

In [54]:
# change the value in the first cell of column 2
# use the at function; pass an index row and column name
df.at[0, 'column 2'] = 1234

# show the DataFrame
df

,column 1,column 2,column 3,col1 + col2,col2 ^ col1
0,1,1234,1000,101,100
1,2,200,2000,202,40000
2,3,300,3000,303,27000000
3,4,400,4000,404,25600000000


[&uarr; Return to Top](#bql-top)

<a id="df-concat"></a>
<h1 style="margin-bottom: 15px;"><span style="color: orange;">Concatenate, Merge, and Pivot</span></h1>
<p>Parsing some BQL requests will require you to alter the structure of the DataFrame. The concatenate, merge, and pivot functions of the <code>pandas</code> library could be useful tools depending on the query.</p>
<ul>
    <li><code>pd.concat()</code> - combine two or more DataFrames by either rows or columns.</li>
    <li><code>pd.DataFrame.merge()</code> - join two DataFrames together based on values in a common column</li>
    <li><code>pd.DataFrame.pivot()</code> - reshape a DataFrame using its values</li>
</ul>

<h4 style="color: orange; margin-bottom: 10px;">Concatenate DataFrame Columns</h4>
<p>Read more about the <code>pd.concat()</code> function in the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html">pandas docs</a>.</p>

In [55]:
# declare two DataFrames
df1 = pd.DataFrame(data={'df1_col1': [1, 2, 3],
                         'df1_col2': [4, 5, 6]})

df2 = pd.DataFrame(data={'df2_col1': [7, 8, 9],
                         'df2_col2': [10, 11, 12]})

# concatenate the two DataFrames
# pass a list of DataFrames to concatenate
# use axis=1 to indicate that you are joining on the columns
df3 = pd.concat([df1, df2], axis=1)

# show the DataFrame
df3


,df1_col1,df1_col2,df2_col1,df2_col2
0,1,4,7,10
1,2,5,8,11
2,3,6,9,12


<h4 style="color: orange;">Concatenate DataFrame Rows</h4>
<p>Read more about the <code>pd.concat()</code> function in the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html">pandas docs</a>.</p>

In [56]:
# declare two DataFrames
df1 = pd.DataFrame(data={'column 1': [1, 2, 3],
                         'column 2': [7, 8, 9]})

df2 = pd.DataFrame(data={'column 1': [4, 5, 6],
                         'column 2': [10, 11, 12]})

# concatenate the two DataFrames
# pass a list of DataFrames to concatenate
# use axis=0 to indicate that you are joining rows of the columns with the same name
df3 = pd.concat([df1, df2], axis=0)

# show the DataFrame
df3


,column 1,column 2
0,1,7
1,2,8
2,3,9
0,4,10
1,5,11
2,6,12


<h4 style="color: orange;">Merge DataFrames</h4>
<p>Read more about the <code>pd.DataFrame.merge()</code> function in the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html">pandas docs</a>.</p>

In [57]:
# declare two DataFrames
df1 = pd.DataFrame(data={'column 1': [1, 2, 3],
                         'column 2': [7, 8, 9]})

df2 = pd.DataFrame(data={'column 1': [1, 2, 100],
                         'column 2': [10, 11, 12]})

# merge the two DataFrames on column 1
# use an 'inner' join to only keep rows that have common column 1 values
df3 = df1.merge(df2, on='column 1', how='inner', suffixes=['_df1', '_df2'])

# show the DataFrame
df3


,column 1,column 2_df1,column 2_df2
0,1,7,10
1,2,8,11


<h4 style="color: orange; margin-bottom: 10px;">Pivot a DataFrame</h4>
<p>The example below is a common workflow for parsing a single time series dataset requested for multiple universes. Our DataFrame <code>df</code> contains three days of closing prices for the stocks of both AAPL and IBM. The raw response from BQL will place the rows for the two requests on top of each other. For analysis purposes, we want each ticker in its own column and each date to have its own row. The pivot function of the <code>pandas</code> library can help us accomplish this.</p>
<p>Read more about the <code>pd.DataFrame.pivot()</code> function in the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html">pandas docs</a>.</p>

In [58]:
# declare two DataFrames
df = pd.DataFrame(data={'Closing Prices': [50.00, 50.01, 50.02, 100.0, 101.0, 105.0],
                        'Date': ['Jan 1', 'Jan 2', 'Jan 3', 'Jan 1', 'Jan 2', 'Jan 3']},
                  index=['AAPL', 'AAPL', 'AAPL', 'IBM', 'IBM', 'IBM'])
df.index.name = 'ID'

# show the DataFrame before a pivot
print('Before Pivot:')
print(df)

# pivot the DataFrame
# first we reset the index to get 'ID' into a column
pivot_df = df.reset_index()

# pivot the dataframe so that each ticker has its own column
# each date should have its own row
pivot_df = pivot_df.pivot(index='Date', values='Closing Prices', columns='ID')

# show the DataFrame
print('\n\nAfter Pivot:')
pivot_df

Before Pivot:
      Closing Prices   Date
ID                         
AAPL           50.00  Jan 1
AAPL           50.01  Jan 2
AAPL           50.02  Jan 3
IBM           100.00  Jan 1
IBM           101.00  Jan 2
IBM           105.00  Jan 3


After Pivot:


ID,AAPL,IBM
Date,,
Jan 1,50.00,100.0
Jan 2,50.01,101.0
Jan 3,50.02,105.0


The `pandas` library is an extensive topic, and we've only covered the basics in this notebook. The library is open-source, so all the documentation is available online. See [pandas.pydata.org/docs/](https://pandas.pydata.org/docs/) for additional information, and if you're having trouble with a specific issue, check [StackOverflow](https://stackoverflow.com/questions/tagged/pandas) to see if another developer

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice with DataFrames and the <code>pandas</code> library.
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Import the <code>pandas</code> library.</li>
            <li>Declare a variable named <code>my_df</code> and set it equal to a DataFrame with two columns of numerical data.</li>
            <li>Add a third column whose values are the sum of the values in each row of the first two columns.</li>
            <li>Declare a second DataFrame named <code>my_df2</code> with three columns using the same column names as <code>my_df</code>.</li>
            <li>Use the <code>pd.concat()</code> function to create a third DataFrame <code>my_df3</code>, also with three columns, and combining all the rows of <code>my_df</code> and <code>my_df2</code>.</li>
            <li>Give <code>my_df3</code> a new index by calling <code>my_df3.index = </code> and set it equal to a list of unique values.</li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# Step 2
import pandas as pd

# Step 3
my_df = pd.DataFrame(data={'col1': [1, 2, 3], 
                           'col2': [4, 5, 6]})

# Step 4
my_df['col3'] = my_df['col1'] + my_df['col2']

# Step 5
my_df2 = pd.DataFrame(data={'col1': [100, 200, 300],
                            'col2': [400, 500, 600],
                            'col3': [700, 800, 900]})

# Step 6
my_df3 = pd.concat([my_df, my_df2])

# Step 7
my_df3.index = list(range(len(my_df3)))  # <-- set the index to a list of values ranging from zero to the length of the DataFrame

# Show DataFrame
my_df3

[&uarr; Return to Top](#bql-top)

---
<a id="bql-editor"></a>
<h1>Using the BQL Editor Tool<span style="color: orange;"><br>Search Fields, Universes, and Functions</span></h1>

BQuant provides a useful tool called the "BQL Editor" to help you write and implement BQL queries. 

The BQL Editor allows you to:
   1. Search for Fields, Functions, and Universes
   1. Run a test query and get a preview of the results
   1. Copy the query's Python code so you can paste it in a Jupyter cell
   
You can access the BQL Editor in the toolbar at the top of the screen. Clicking the BQL Editor button will open up a separate tab in which you can write your test query.

The example below demonstrates the use of the BQL Editor in building a query for the most recent S&P 500 price.

<img src="../img/Boot Camp/bql_editor.gif">

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice with the BQL Editor
        <ol>
            <li>Add a new Jupyter code cell below this one.</li>
            <li>Click into the BQL Editor and build a query for the latest daily high <code>PX_HIGH()</code> for <code>'AAPL US Equity'</code>.</li>
            <li>Preview the results of the query with the Preview button in the toolbar.</li>
            <li>Copy the Python code for the query by selecting Copy &rarr; BQL Object Model.</li>
            <li>Paste the code in the Jupyter cell.</li>
        </ol>
    </p>
    <br>
</div>

[&uarr; Return to Top](#bql-top)

---
<a id="bql-example"></a>
<h1>BQL Example<span style="color: orange;"><br>One Universe, One Data Item</span></h1>

Get one time series dataset (daily closing prices) for one universe (S&P 500 Index).

In [3]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Use BQL to Get Data
universe = 'BWORLDPR Index'
date_range = bq.func.range('2019-10-01', '2019-12-31')        # <-- use bq.func.range() to specify a range of dates
closing_prices = bq.data.px_last(dates=date_range).dropna()   # <-- use the dropna() BQL function to drop non-trading days
req = bql.Request(universe, {'Closing Price': closing_prices})
res = bq.execute(req)
df = res[0].df()

# use the head() function to display the first 5 rows
df.head()


,DATE,CURRENCY,Closing Price
ID,,,
BWORLDPR Index,2019-10-01,USD,182.33
BWORLDPR Index,2019-10-02,USD,181.88
BWORLDPR Index,2019-10-03,USD,180.92
BWORLDPR Index,2019-10-04,USD,181.19
BWORLDPR Index,2019-10-07,USD,181.11


[&uarr; Return to Top](#bql-top)

---
<a id="bql-example2"></a>
<h1>BQL Example<span style="color: orange;"><br>One Universe, Multiple Data Items</span></h1>

Get two time series datasets (daily price and volume) for one universe (S&P 500 Index). Use `pd.DataFrame.merge()` to join the results.

In [4]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Use BQL to Get Data
universe = 'BWORLDPR Index'
date_range = bq.func.range('2019-10-01', '2019-12-31')
closing_prices = bq.data.px_last(dates=date_range).dropna()
vol = bq.data.px_volume(dates=date_range).dropna()                    # <-- use the same date range for the volume series
req = bql.Request(universe, {'Closing Price': closing_prices,
                             'Volume': vol})
res = bq.execute(req)

# Step 3: Additional Data Munging
# extract each dataframe
price_df = res[0].df()
vol_df = res[1].df()

# merge the two dataframes together on the DATE column
df = price_df.merge(vol_df, how='inner', on='DATE', left_index=True)   # <-- left_index=True will keep the index from price_df

# use the head() function to display the first 5 rows
df.head()


,DATE,CURRENCY,Closing Price,Volume
ID,,,,
BWORLDPR Index,2019-10-01,USD,182.33,4.166384e+09
BWORLDPR Index,2019-10-02,USD,181.88,6.545258e+09
BWORLDPR Index,2019-10-03,USD,180.92,7.365833e+09
BWORLDPR Index,2019-10-04,USD,181.19,7.333393e+09
BWORLDPR Index,2019-10-07,USD,181.11,3.567906e+09


[&uarr; Return to Top](#bql-top)

---
<a id="bql-example3"></a>
<h1>BQL Example<span style="color: orange;"><br>Multiple Universes, One Data Item</span></h1>

Get one dataset (daily prices) for multiple universes (AAPL and IBM). Use `pd.DataFrame.pivot()` to reshape the DataFrame.

In [61]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# bq = bql.Service()

# Step 2: Use BQL to Get Data
universe = ['AAPL US Equity', 'IBM US Equity']                          # <-- use a list of strings as the universe
date_range = bq.func.range('2019-10-01', '2019-12-31')
closing_prices = bq.data.px_last(dates=date_range, per='D').dropna()
req = bql.Request(universe, {'Closing Price': closing_prices})
res = bq.execute(req)

# Step 3: Additional Data Munging
df = res[0].df().reset_index().pivot(index='DATE', values='Closing Price', columns='ID')     # <-- parse and pivot the dataframe in one line

# use the head() function to display the first 5 rows
df.head()


ID,AAPL US Equity,IBM US Equity
DATE,,
2019-10-01,56.1475,143.66
2019-10-02,54.7400,141.69
2019-10-03,55.2050,142.02
2019-10-04,56.7525,142.99
2019-10-07,56.7650,141.28


[&uarr; Return to Top](#bql-top)

---
<a id="bql-example4"></a>
<h1>BQL Example<span style="color: orange;"><br>Multiple Universes, Multiple Data Items</span></h1>

Get two datasets (daily price and volume) for multiple universes (AAPL and IBM). Use `pd.DataFrame.merge()` to join the DataFrames and then `pd.DataFrame.pivot()` to reshape.

Read more about the `pandas.MultiIndex` object used in this example in the [pandas docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html).

In [62]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
# import bql
# import pandas as pd
# bq = bql.Service()

# Step 2: Use BQL to Get Data
universe = ['AAPL US Equity', 'IBM US Equity']
date_range = bq.func.range('2019-10-01', '2019-12-31')
closing_prices = bq.data.px_last(dates=date_range, per='D').dropna()
vol = bq.data.px_volume(dates=date_range).dropna()
req = bql.Request(universe, {'Closing Price': closing_prices,
                             'Volume': vol})
res = bq.execute(req)

# Step 3: Additional Data Munging
# pivot each result
price_df = res[0].df().reset_index().pivot(index='DATE', columns='ID', values='Closing Price')
vol_df = res[1].df().reset_index().pivot(index='DATE', columns='ID', values='Volume')

# reassign the columns with the additional label of price or volume
price_df.columns = pd.MultiIndex.from_product([['Closing Price'], list(price_df.columns)])
vol_df.columns = pd.MultiIndex.from_product([['Volume'], list(vol_df.columns)])

# merge the two dataframes by common date
df = price_df.merge(vol_df, how='inner', on='DATE')

# use the head() function to display the first 5 rows
df.head()



Closing Price                       Volume              
           AAPL US Equity IBM US Equity AAPL US Equity IBM US Equity
DATE                                                                
2019-10-01        56.1475        143.66    144748652.0     2973714.0
2019-10-02        54.7400        141.69    143069028.0     3347276.0
2019-10-03        55.2050        142.02    121410744.0     3161718.0
2019-10-04        56.7525        142.99    139022212.0     2449483.0
2019-10-07        56.7650        141.28    123557076.0     2488799.0

[&uarr; Return to Top](#bql-top)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="2 Python Basics.ipynb">&larr; Back to Python Basics</a>&emsp;&emsp;
    <a href="#bql-top">&uarr; Return to Top</a>&emsp;&emsp;
    <a href="4 Boot Camp Project.ipynb">Continue to Boot Camp: Final Project &rarr;</a>
    <br>
    <br>
    <a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>
</p>